# Preprocessing

In [25]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use('ggplot')

In [26]:
merged_df = pd.read_csv("../00_data/1_interim/merged_data.csv", index_col=["Code", "Year"])

### Handle High Missing Values by Code (Country ID)

In [24]:
# Missing Values by Country ID (Code)
all_missing_values = merged_df.isnull().groupby(level=0).sum()
all_missing_values["total_missing"] = all_missing_values.sum(axis=1)
all_missing_values["total_missing_%"] = round( ( all_missing_values["total_missing"] / 54 ) * 100, 2)
#plot top 12 countries with most missing values
top_missing_countries = all_missing_values.sort_values(ascending=False, by="total_missing")
top_12_missing = top_missing_countries["total_missing"].head(12)
print(f"Top 12 Countries with most missing values:\n{top_12_missing}")
#Threshold 50%: all countries with missing values over 50% get excluded from Dataframe
# 9 x 6 = 54 data point for each country --> 27 NaNs as upper limit: 
# AIA       49 (Anguilla)
# MSR       48 (Montserrat)
# OWID_KOS  48 (Kosovo)
# COK       42 (Cook Islands)
# NIU       42 (Niue)
# TCA       39 (Turks and Caicos Islands)
# VGB       38 (British Virgin Islands)

Top 12 Countries with most missing values:
Code
AIA         49
MSR         48
OWID_KOS    48
COK         42
NIU         42
TCA         39
VGB         38
PRK         26
SSD         26
MCO         25
GNQ         22
VEN         21
Name: total_missing, dtype: int64


In [4]:
# Exclude high missing values countries (threshold >= 50%) & save as new df
# new filtered version has now 967 missing values (306 less than before)
exclude_countries = top_missing_countries[top_missing_countries["total_missing_%"] >= 50]
filtered_df = merged_df[~merged_df.index.get_level_values(0).isin(exclude_countries.index.tolist())].copy()
filtered_df.to_csv('../00_data/1_interim/filtered_data_01.csv', index=True)
filtered_df.isna().sum().sum()

np.int64(967)

### Handle Missing Values for "vaccination_coverage_who_unicef"

In [5]:
filtered_df.isna().sum()

Entity                                        0
child_mortality_igme                          0
annual_healthcare_expenditure_per_capita     16
gdp_per_capita_worldbank                     42
nurses_and_midwives_per_1000_people         300
physicians_per_1000_people                  405
prevalence_of_undernourishment              156
share_of_population_urban                     0
share_without_improved_water                 16
vaccination_coverage_who_unicef               6
years_of_schooling                           26
dtype: int64

In [6]:
#vaccination_coverage_who_unicef (3 doses diphtheria, tetanus and pertussis vaccine) has only 6 missing values for country NIC (Nicaragua)
filtered_df.loc[filtered_df["vaccination_coverage_who_unicef"].isnull(), ["vaccination_coverage_who_unicef"]]

vaccination_coverage_who_unicef
Code Year                                 
NIC  2013                              NaN
     2014                              NaN
     2015                              NaN
     2016                              NaN
     2017                              NaN
     2018                              NaN

In [7]:
# OWID has data from WHO, but WUENIC data source does not exist for Nicaragua
# Source: WHO immunizationdata, https://immunizationdata.who.int/dashboard/regions/region-of-the-americas/NIC (Zugriff am: 27.10.2025)
# Use worldbank countries groups (7 different groups) & imputate vaccination values for "NIC" by median of its countrys group
world_regions = pd.read_csv("../00_data/1_interim/world-regions-worldbank.csv")
country_groups = world_regions.loc[:, ["Code", "World regions according to WB"]]

new_col_group = filtered_df.copy()
new_col_group = new_col_group.reset_index()
new_col_group = new_col_group.merge(country_groups, on=['Code'], how="left")
new_col_group =  new_col_group.set_index(["Code", "Year"])

In [8]:
# Nicaragua (NIC) belongs to world region group: "Latin America and Caribbean (WB)"
# Calculate median of column vaccination_coverage for "Latin America and Caribbean (WB)"
# set value to missing values for Nicaragua (NIC)

#latin_caribbean = new_col_group.loc[new_col_group["World regions according to WB"] == "Latin America and Caribbean (WB)"]
#latin_caribbean.head()
latin_caribbean = new_col_group.groupby(["World regions according to WB"])["vaccination_coverage_who_unicef"].median()
latin_caribbean

World regions according to WB
East Asia and Pacific (WB)                                  92.0
Europe and Central Asia (WB)                                95.5
Latin America and Caribbean (WB)                            92.0
Middle East, North Africa, Afghanistan and Pakistan (WB)    97.0
North America (WB)                                          92.5
South Asia (WB)                                             98.0
Sub-Saharan Africa (WB)                                     85.0
Name: vaccination_coverage_who_unicef, dtype: float64

In [9]:
#new_col_group.loc[new_col_group["Entity"] == "Nicaragua", "vaccination_coverage_who_unicef"] = 92.0

In [10]:
filtered_df.loc[filtered_df["Entity"] == "Nicaragua", "vaccination_coverage_who_unicef"] = 92.0

In [11]:
filtered_df.isna().sum()

Entity                                        0
child_mortality_igme                          0
annual_healthcare_expenditure_per_capita     16
gdp_per_capita_worldbank                     42
nurses_and_midwives_per_1000_people         300
physicians_per_1000_people                  405
prevalence_of_undernourishment              156
share_of_population_urban                     0
share_without_improved_water                 16
vaccination_coverage_who_unicef               0
years_of_schooling                           26
dtype: int64

In [12]:
filtered_df.loc[filtered_df["Entity"] == "Nicaragua"]

Entity  child_mortality_igme  \
Code Year                                    
NIC  2013  Nicaragua             20.337691   
     2014  Nicaragua             19.446352   
     2015  Nicaragua             18.606286   
     2016  Nicaragua             17.805850   
     2017  Nicaragua             17.066324   
     2018  Nicaragua             16.364048   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
NIC  2013                                 356.79343                 6494.1494   
     2014                                 399.16168                 6712.2173   
     2015                                 426.61290                 6939.0327   
     2016                                 463.54483                 7158.7580   
     2017                                 505.41940                 7391.1970   
     2018                                 503.79822                 7049.0273   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
NIC  2013                                1.347                       0.890   
     2014                                1.341                       0.885   
     2015                                1.350                       0.920   
     2016                                1.447                       0.941   
     2017                                1.530                       0.975   
     2018                                  NaN                       0.664   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                                              
NIC  2013                            19.3                     57.505   
     2014                            19.8                     57.700   
     2015                            19.3                     57.895   
     2016                            18.3                     58.090   
     2017                            17.6                     58.299   
     2018                            17.6                     58.522   

           share_without_improved_water  vaccination_coverage_who_unicef  \
Code Year                                                                  
NIC  2013                     17.447060                             92.0   
     2014                     17.334541                             92.0   
     2015                     17.220947                             92.0   
     2016                     17.106255                             92.0   
     2017                     17.033340                             92.0   
     2018                     16.955544                             92.0   

           years_of_schooling  
Code Year                      
NIC  2013            8.581959  
     2014            8.781239  
     2015            8.980518  
     2016            9.135801  
     2017            9.291083  
     2018            9.446366

In [117]:
# leave world groups with One Hot Encoding? 
#new_col_group.head(2)

In [21]:
#filtered_df.loc[:, ["annual_healthcare_expenditure_per_capita", "share_without_improved_water"]]
filtered_df[filtered_df['annual_healthcare_expenditure_per_capita'].isna()]

Entity  child_mortality_igme  \
Code Year                                      
PRK  2013  North Korea             24.113420   
     2014  North Korea             22.628267   
     2015  North Korea             21.352856   
     2016  North Korea             20.605180   
     2017  North Korea             19.118627   
     2018  North Korea             18.416384   
SSD  2013  South Sudan             99.224690   
     2014  South Sudan            182.331310   
     2015  South Sudan            236.565440   
     2016  South Sudan            263.925670   
VEN  2013    Venezuela             17.908800   
     2014    Venezuela             18.282752   
     2015    Venezuela             19.205246   
     2016    Venezuela             24.262680   
     2017    Venezuela             24.258077   
     2018    Venezuela             24.257355   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
PRK  2013                                       NaN                       NaN   
     2014                                       NaN                       NaN   
     2015                                       NaN                       NaN   
     2016                                       NaN                       NaN   
     2017                                       NaN                       NaN   
     2018                                       NaN                       NaN   
SSD  2013                                       NaN                       NaN   
     2014                                       NaN                       NaN   
     2015                                       NaN                       NaN   
     2016                                       NaN                       NaN   
VEN  2013                                       NaN                       NaN   
     2014                                       NaN                       NaN   
     2015                                       NaN                       NaN   
     2016                                       NaN                       NaN   
     2017                                       NaN                       NaN   
     2018                                       NaN                       NaN   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
PRK  2013                                  NaN                         NaN   
     2014                                  NaN                       3.496   
     2015                                  NaN                         NaN   
     2016                                  NaN                       3.689   
     2017                                4.434                       3.671   
     2018                                  NaN                         NaN   
SSD  2013                                  NaN                         NaN   
     2014                                  NaN                         NaN   
     2015                                  NaN                         NaN   
     2016                                  NaN                         NaN   
VEN  2013                                  NaN                         NaN   
     2014                                  NaN                         NaN   
     2015                                  NaN                         NaN   
     2016                                  NaN                         NaN   
     2017                                1.934                       1.664   
     2018                                2.001                         NaN   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                                              
PRK  2013                            41.8                     60.918   
     2014                            42.7                     61.098   

In [32]:
filtered_df[filtered_df['share_without_improved_water'].isna()]

Entity  child_mortality_igme  \
Code Year                                                
ARG  2017              Argentina             11.034559   
     2018              Argentina             10.745510   
HRV  2013                Croatia              5.052622   
     2014                Croatia              4.974593   
     2015                Croatia              4.911460   
     2016                Croatia              4.859440   
     2017                Croatia              4.812192   
     2018                Croatia              4.767326   
DMA  2018               Dominica             32.118952   
GNQ  2018      Equatorial Guinea             84.172230   
ERI  2017                Eritrea             43.214083   
     2018                Eritrea             41.731744   
GRD  2018                Grenada             18.337375   
KNA  2018  Saint Kitts and Nevis             18.624694   
SAU  2013           Saudi Arabia             10.182618   
     2014           Saudi Arabia              9.587044   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
ARG  2017                                2401.09130                 28334.904   
     2018                                2388.69500                 27367.115   
HRV  2013                                1420.10200                 28401.686   
     2014                                1475.16850                 28460.730   
     2015                                1553.30970                 29471.752   
     2016                                1692.23120                 30824.860   
     2017                                1798.32900                 32351.873   
     2018                                1943.08480                 33730.875   
DMA  2018                                 779.71155                 16877.125   
GNQ  2018                                 482.17950                 20165.480   
ERI  2017                                  69.18159                       NaN   
     2018                                  80.70452                       NaN   
GRD  2018                                 761.46090                 16975.248   
KNA  2018                                1524.73240                 30576.248   
SAU  2013                                2677.10000                 57998.480   
     2014                                3106.07320                 58872.266   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
ARG  2017                                2.593                       3.979   
     2018                                  NaN                         NaN   
HRV  2013                                6.130                       2.995   
     2014                                6.101                       3.105   
     2015                                6.160                       3.156   
     2016                                6.652                       6.394   
     2017                                6.854                       6.624   
     2018                                6.939                       6.766   
DMA  2018                                6.511                       1.116   
GNQ  2018                                0.270                         NaN   
ERI  2017                                1.333                       0.066   
     2018                                1.443                       0.081   
GRD  2018                                5.747                       1.314   
KNA  2018                                  NaN                       3.034   
SAU  2013                                  NaN                       2.159   
     2014                                5.006                       2.140   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                    